# Tutorial: a simple simulation of alanine dipeptide with ANI-2x using OpenMM-Torch and NNPOps

You can run this tutorial directly in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openmm/openmm-torch/blob/master/tutorials/openmm-torch-nnpops.ipynb)

Covered topics:
 * Installation of the software with [Conda](https://docs.conda.io/)
 * Creation of an NNP (neural network potential) with [TorchANI](https://aiqm.github.io/torchani/)
 * Acceleration of [ANI-2x](https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/60c747e9567dfec574ec48df/original/extending-the-applicability-of-the-ani-deep-learning-molecular-potential-to-sulfur-and-halogens.pdf) with [NNPOps](https://github.com/openmm/NNPOps) 
 * Integration of [OpenMM](https://openmm.org/) and [PyTorch](https://pytorch.org/) and with [OpenMM-Torch](https://github.com/openmm/openmm-torch)
 * Setup and execution of a simulation

## Install Conda

[Conda](https://docs.conda.io/) is a package and environment manager. On Google Colab, Conda is installed with [conda-colab](https://github.com/jaimergp/condacolab). On your computer, you should follow these [installation instructions](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).

⚠️ Do not use conda-colab on your computer!

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:18
🔁 Restarting kernel...


## Install software

The [conda-forge](https://conda-forge.org/) channel is used for software.

⚠️ The installation might take up to 10 min!

In [2]:
# Note: at the moment only the beta releases work correctly.
!mamba install -q \
               -c conda-forge/label/openmm_rc \
               -c conda-forge/label/openmm-torch_rc \
               -c conda-forge \
               openmm==8.0.0beta \
               openmm-torch==1.0beta \
               nnpops==0.2 \
               torchani==2.2.2 \
               openmmtools \
               &> /dev/null # Comment this line to see a log

## Prepare a simulation system

For simplicity, the alanine dipeptide system from [OpenMM-Tools](https://openmmtools.readthedocs.io/en/latest/) is used.

In [1]:
import openmmtools

# Get the system of alanine dipeptide
ala2 = openmmtools.testsystems.AlanineDipeptideVacuum(constraints=None)

# Remove MM forces
while ala2.system.getNumForces() > 0:
  ala2.system.removeForce(0)

# The system should not contain any additional force and constrains
assert ala2.system.getNumConstraints() == 0
assert ala2.system.getNumForces() == 0

# Get the list of atomic numbers
atomic_numbers = [atom.element.atomic_number for atom in ala2.topology.atoms()]

## Create a NNP

A NNP (neural network potential) is created with [TorchANI](https://aiqm.github.io/torchani/). In this case [ANI-2x](https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/60c747e9567dfec574ec48df/original/extending-the-applicability-of-the-ani-deep-learning-molecular-potential-to-sulfur-and-halogens.pdf) is used, which can be accelerated with [NNPOPs](https://github.com/openmm/NNPOps).

In [2]:
import torch as pt
from torchani.models import ANI2x
from NNPOps import OptimizedTorchANI

class NNP(pt.nn.Module):

  def __init__(self, atomic_numbers):

    super().__init__()

    # Store the atomic numbers
    self.atomic_numbers = pt.tensor(atomic_numbers).unsqueeze(0)

    # Create an ANI-2x model
    self.model = ANI2x(periodic_table_index=True)

    # Accelerate the model
    self.model = OptimizedTorchANI(self.model, self.atomic_numbers)

  def forward(self, positions):

    # Prepare the positions
    positions = positions.unsqueeze(0).float() * 10 # nm --> Å
    
    # Run ANI-2x
    result = self.model((self.atomic_numbers, positions))
    
    # Get the potential energy
    energy = result.energies[0] * 2625.5 # Hartree --> kJ/mol

    return energy

# Create an instance of the model
nnp = NNP(atomic_numbers)

/usr/local/lib/python3.7/site-packages/torchani/resources/


At this point, the potential energy of the system can be computed with the NNP:

In [3]:
# Comute the potential energy
pos = pt.tensor(ala2.positions.tolist())
energy_1 = float(nnp(pos))
print(energy_1)

# Check if the energy is correct
assert pt.isclose(pt.tensor(energy_1), pt.tensor(-1301523.8703817206))

-1301523.8702643516


## Add the NNP to the system

In order to use the NNP in a simulation, it has to loaded with [OpenMM-Torch](https://github.com/openmm/openmm-torch) and added to the system.

In [4]:
from openmmtorch import TorchForce

# Save the NNP to a file and load it with OpenMM-Torch
pt.jit.script(nnp).save('model.pt')
force = TorchForce('model.pt')

# Add the NNP to the system
ala2.system.addForce(force)
assert ala2.system.getNumForces() == 1

## Setup a simulation

Setup a simulation with [OpenMM](https://openmm.org/).

In [5]:
import sys
from openmm import LangevinMiddleIntegrator
from openmm.app import Simulation, StateDataReporter
from openmm.unit import kelvin, picosecond, femtosecond

# Create an integrator with a time step of 1 fs
temperature = 298.15 * kelvin
frictionCoeff = 1 / picosecond
timeStep = 1 * femtosecond
integrator = LangevinMiddleIntegrator(temperature, frictionCoeff, timeStep)

# Create a simulation and set the initial positions and velocities
simulation = Simulation(ala2.topology, ala2.system, integrator)
simulation.context.setPositions(ala2.positions)
simulation.context.setVelocitiesToTemperature(temperature)

# Configure a reporter to print to the console every 0.1 ps (100 steps)
reporter = StateDataReporter(file=sys.stdout, reportInterval=100, step=True, time=True, potentialEnergy=True, temperature=True)
simulation.reporters.append(reporter)

At this point, the potential energy of the system can be computed again:

In [6]:
from openmm.unit import kilojoule_per_mole

# Comute the potential energy
state = simulation.context.getState(getEnergy=True)
energy_2 = state.getPotentialEnergy().value_in_unit(kilojoule_per_mole)
print(energy_2)

# Check if the energy is correct
assert pt.isclose(pt.tensor(energy_1), pt.tensor(energy_2))

-1301523.8702643516


## Run the simulation

Run your first NNP simulation.

⚠️ The simulations are not deterministic! Each time the log will be a bit different.

In [7]:
# Run the simulations for 1 ps (1000 steps)
simulation.step(1000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
100,0.10000000000000007,-1301523.916390367,92.40824830382874
200,0.20000000000000015,-1301518.7990238892,119.2893445291452
300,0.3000000000000002,-1301513.695350461,152.9633322198495
400,0.4000000000000003,-1301508.793747321,161.74950978774066
500,0.5000000000000003,-1301504.6699876415,164.00853843792856
600,0.6000000000000004,-1301498.9500878605,159.56241762805863
700,0.7000000000000005,-1301508.518243164,197.16748430772566
800,0.8000000000000006,-1301501.4794679733,217.2560582437653
900,0.9000000000000007,-1301479.1362451615,189.695421119118
1000,1.0000000000000007,-1301481.766797349,256.3789993541881
